In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

In [ ]:
################################
# Order Book Sim
################################

# Parameters
n_steps = 2000
dt = 1
ref_price_a, ref_price_b = 100, 100
lambda_a, lambda_b = 1.0, 1.0
std_price_a, std_price_b = 1.0, 1.0
delta_a, delta_b = dt * 0.01, dt * 0.01
phi_a, phi_b = dt * 20, dt * 20
initial_particles_a = 500
initial_particles_b = 500
n_histograms = 1250

# Initialize particles
prices_a = np.random.normal(loc=ref_price_a, scale=std_price_a, size=initial_particles_a)
mask_a = prices_a[:] > ref_price_a + 0.5
prices_a = prices_a[mask_a]
sizes_a = np.random.exponential(scale=1 / lambda_a, size=len(prices_a))
A_initial = prices_a.copy()

prices_b = np.random.normal(loc=ref_price_b, scale=std_price_b, size=initial_particles_b)
mask_b = prices_b[:] < ref_price_b - 0.5
prices_b = prices_b[mask_b]
sizes_b = np.random.exponential(scale=1 / lambda_b, size=len(prices_b))
B_initial = prices_b.copy()

particles_a = np.column_stack((prices_a, sizes_a))
particles_b = np.column_stack((prices_b, sizes_b))

starting_distribution_a = particles_a.copy()
starting_distribution_b = particles_b.copy()

price_bins = np.linspace(
    min(ref_price_a, ref_price_b) - 5 * std_price_b,
    max(ref_price_a, ref_price_b) + 5 * std_price_a,
    100
)

hist_a, _ = np.histogram(A_initial, bins=price_bins)
hist_b, _ = np.histogram(B_initial, bins=price_bins)

# Arrays to store statistics
time_steps = []
n_a_array = []
n_b_array = []
mean_price_a_array = []
mean_price_b_array = []
std_price_a_array = []
std_price_b_array = []

# To store histograms of the last 1000 steps
last_histograms_a = []
last_histograms_b = []

# Functions
def add_entries():
    global particles_a, particles_b

    n_new_a = np.random.poisson(phi_a)
    n_new_b = np.random.poisson(phi_b)

    # Generate all new A and B particles
    new_prices_a = np.random.normal(loc=ref_price_a, scale=std_price_a, size=n_new_a)
    new_sizes_a = np.random.exponential(scale=1 / lambda_a, size=n_new_a)
    
    new_prices_b = np.random.normal(loc=ref_price_b, scale=std_price_b, size=n_new_b)
    new_sizes_b = np.random.exponential(scale=1 / lambda_b, size=n_new_b)

    # Create a list of all new particles with their type info
    all_new_particles = []
    
    for p, s in zip(new_prices_a, new_sizes_a):
        all_new_particles.append(('A', p, s))
        
    for p, s in zip(new_prices_b, new_sizes_b):
        all_new_particles.append(('B', p, s))

    # Shuffle to randomize processing order
    np.random.shuffle(all_new_particles)

    # Process each particle in the shuffled list
    for particle in all_new_particles:
        ptype, price, size = particle
        
        if ptype == 'A':
            # Try to insert A particle: must be <= max price in B
            if len(particles_b) == 0:
                # No B particles available to remove; skip adding
                continue
                
            max_price_idx = np.argmax(particles_b[:, 0])
            price_b, size_b = particles_b[max_price_idx]
            
            if price <= price_b:
                particles_b = np.delete(particles_b, max_price_idx, axis=0)
            else:
                particles_a = np.vstack((particles_a, [price, size]))
                
        elif ptype == 'B':
            # Try to insert B particle: must be >= min price in A
            if len(particles_a) == 0:
                # No A particles available to remove; skip adding
                continue
                
            min_price_idx = np.argmin(particles_a[:, 0])
            price_a, size_a = particles_a[min_price_idx]
            
            if price_a <= price:
                particles_a = np.delete(particles_a, min_price_idx, axis=0)
            else:
                particles_b = np.vstack((particles_b, [price, size]))
    
def apply_cancellations():
    global particles_a, particles_b

    particles_a = particles_a[np.random.rand(len(particles_a)) > delta_a]
    particles_b = particles_b[np.random.rand(len(particles_b)) > delta_b]

def record_statistics(step):
    global time_steps, n_a_array, n_b_array, mean_price_a_array, mean_price_b_array, std_price_a_array, std_price_b_array, particles_a, particles_b

    time_steps.append(step)
    n_a_array.append(len(particles_a))
    n_b_array.append(len(particles_b))

def record_histograms(n_histograms = n_histograms):
    global last_histograms_a, last_histograms_b
    
    hist_a, _ = np.histogram(particles_a[:, 0], bins=price_bins)
    hist_b, _ = np.histogram(particles_b[:, 0], bins=price_bins)
    
    last_histograms_a.append(hist_a)
    last_histograms_b.append(hist_b)

    if len(last_histograms_a) > n_histograms:
        last_histograms_a.pop(0)
    if len(last_histograms_b) > n_histograms:
        last_histograms_b.pop(0)

def one_step(step):
    record_statistics(step)
    add_entries()
    apply_cancellations()
    if step >= n_steps - n_histograms:
        record_histograms()

# Run simulation
for step in range(n_steps):
    one_step(step)
    if len(particles_a) == 0 and len(particles_b) == 0:
        print(f"Simulation ended at step {step}.")
        break

# Average histograms
dp = price_bins[1]-price_bins[0]
final_counts_a = np.mean(last_histograms_a/dp, axis=0)
final_counts_b = np.mean(last_histograms_b/dp, axis=0)

initial_counts_a, _ = np.histogram(starting_distribution_a[:, 0], bins=price_bins)
initial_counts_b, _ = np.histogram(starting_distribution_b[:, 0], bins=price_bins)
xvalues = [(price_bins[i] + price_bins[i+1])/2 for i in range(len(price_bins) -1 )]

In [ ]:
# Define parameters 
phi_a = 20.0
phi_b = 20.0
Delta_a = 0.01
Delta_b = 0.01
gamma_a = 0
gamma_b = 0
ref_price_a = 100
ref_price_b = 100
std_price_a = 1
std_price_b = 1

In [ ]:
# Set Seaborn style and font settings
sns.set(style="ticks", context="paper")
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['mathtext.fontset'] = 'cm'  # Use LaTeX-style math fonts
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['legend.fontsize'] = 14
mpl.rcParams['lines.linewidth'] = 2

def plot(n_a_initial, n_b_initial, na_eq_dist, nb_eq_dist, n_a, n_b):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot Metropolis final distributions
    ax.plot(xvalues, n_a, 
            alpha=0.9, label=r'$n_a(p)$ reached distribution', color='firebrick', linestyle='--', linewidth=3)
    ax.plot(xvalues, n_b, 
            alpha=0.9, label=r'$n_b(p)$ reached distribution', color='navy', linestyle='--', linewidth=3)

    # Plot Master Equation equilibrium distributions
    ax.plot(xvalues, na_eq_dist, 
            alpha=0.7, label=r'$n_a(p)$ target distribution', color='darkorange', linestyle='-', linewidth=2.5)
    ax.plot(xvalues, nb_eq_dist, 
            alpha=0.7, label=r'$n_b(p)$ target distribution', color='teal', linestyle='-', linewidth=2.5)

    # Plot initial conditions
    ax.plot(xvalues, n_b_initial, 
            alpha=0.7, label=r'$n_a(p)$ and $n_b(p)$ initial distributions', color='black', linestyle='-', linewidth=2)

    ax.set_title('Generalized Functional minimization', fontsize=20)
    ax.set_xlabel('Price', fontsize=16)
    plt.xticks(np.arange(95, 106, step=2))
    ax.set_ylabel('Particle Ammount', fontsize=16)
    ax.set_ylim(top=855)
    ax.legend(loc='upper right', frameon=True, fancybox=True, shadow=False)
    ax.grid(True, linestyle='--', alpha=0.6)
    sns.despine(trim=True)
    plt.tight_layout()

    plt.savefig("images/Conexion/Metropolis gas con psi ingles.pdf", format='pdf', dpi=300, bbox_inches='tight')
    plt.show()

def P_a(p):
    return np.exp(-0.5 * ((p - ref_price_a) / std_price_a) ** 2) / (std_price_a * np.sqrt(2 * np.pi))

def P_b(p):
    return np.exp(-0.5 * ((p - ref_price_b) / std_price_b) ** 2) / (std_price_b * np.sqrt(2 * np.pi))

def metropolis_optimized(n_a_initial, n_b_initial, n_steps, dx, interaction, beta, kappa, temperatura, price_grid):
    global P_a_values, P_b_values

    # Precompute constants 
    phi_a_div = phi_a / (1 - gamma_a)
    Delta_a_div = Delta_a / (1 - gamma_a)
    phi_b_div = phi_b / (1 - gamma_b)
    Delta_b_div = Delta_b / (1 - gamma_b)
    
    # Initialize arrays 
    n_a = n_a_initial.copy()
    n_b = n_b_initial.copy()
    log_n_a = np.log(n_a, dtype=np.float64)
    log_n_b = np.log(n_b, dtype=np.float64)

    exp_list_p = np.exp(temperatura * price_grid, dtype=np.float64)
    exp_list_m = np.exp(-1 * temperatura * price_grid, dtype=np.float64)

    for step in range(n_steps):

        # Perturb n_a
        idx = np.random.randint(len(n_a))
        old_val = n_a[idx]
        delta = np.random.choice([0.95, 1.05])
        new_val = old_val * delta
        new_val = max(new_val, 1e-300)
        delta_n = new_val - old_val
        
        # if interactions
        if interaction:

            sum_n_b_after = np.sum( exp_list_m[idx] * exp_list_p[idx:] * n_b[idx:] , dtype=np.float64) * dp
            delta_term1 = kappa * delta_n * sum_n_b_after * dp

        else:
            delta_term1 = 0
        
        # Compute delta_log using precomputed logs
        new_log_val = np.log(new_val, dtype=np.float64)
        delta_log = new_log_val - log_n_a[idx]
        delta_term2_a = -(phi_a_div * P_a_values[idx] * delta_log - Delta_a_div * delta_n) * dp
        delta_E = delta_term1 + delta_term2_a
        
        # Accept or reject the move
        if delta_E < 0:
            n_a[idx] = new_val
            log_n_a[idx] = new_log_val 
        
        # Perturb n_b
        idx = np.random.randint(len(n_b))
        old_val = n_b[idx]
        delta = np.random.choice([0.95, 1.05])
        new_val = old_val * delta
        new_val = max(new_val, 1e-300)
        delta_n = new_val - old_val
        
        if interaction:

            sum_n_a_before = np.sum( exp_list_p[idx] * exp_list_m[:idx+1] * n_a[:idx+1], dtype=np.float64 ) * dp
            delta_term1 = kappa * delta_n * sum_n_a_before * dp
        else:
            delta_term1 = 0
        
        # Compute delta_log 
        new_log_val = np.log(new_val, dtype=np.float64)
        delta_log = new_log_val - log_n_b[idx]
        delta_term2_b = -(phi_b_div * P_b_values[idx] * delta_log - Delta_b_div * delta_n) * dp
        delta_E = delta_term1 + delta_term2_b
        
        # Accept or reject the move
        if delta_E < 0:
            n_b[idx] = new_val
            log_n_b[idx] = new_log_val 
    
    return n_a, n_b

In [ ]:
n_steps = 700000
dx = 0.5
interaction = True

beta = 1000
temperatura_attachment = 20
kappa = 1e10

p_grid = np.array(xvalues) - 100

P_a_values = P_a(np.array(xvalues))
P_b_values = P_b(np.array(xvalues)) 

n_a_eq_dist = final_counts_a 
n_b_eq_dist = final_counts_b

n_a_initial = 1000 * P_a_values
n_b_initial = 1000 * P_a_values

def replace_zeros_with_small_value(array):

    modified_array = array.copy().astype(np.float64)
    modified_array += 10 

    return modified_array

n_a_initial = replace_zeros_with_small_value(n_a_initial)
n_b_initial = replace_zeros_with_small_value(n_b_initial)

n_a, n_b = metropolis_optimized(n_a_initial, n_b_initial, n_steps, dx, interaction,beta, kappa, temperatura_attachment, p_grid)

plot(n_a_initial, n_b_initial, n_a_eq_dist, n_b_eq_dist, n_a, n_b)